#### Grid class

In [1]:
# Class cell -> its the class of all the nodes in the grid, Each cell of the grid is defined as a object of type 
# cell.
class cell:
    def __init__(self,x,y): # Constructor for class cell
        self.x = x # x coordinate of the cell
        self.y = y # y coordinate of the cell
        self.g = 0 # g value of the cell (the distance from start cell to curr cell)
        self.h = 0 # h value of the cell (estimated value of the distance between curr cell and goal cell)
        self.f = 0 # f value of the cell (f = g+h in A*, defines the priority of a node)
        self.blocked = False # Flag used to tell weather a cell is blocked or not
        self.terrain = None # If the cell is unblocked this flag stores the terrain type of the cell it is either of hilly, flat or forest.
        self.false_neg_rate = 0 # Depending on the terrain this variable stores the false negative rate of the cell
        self.prob_of_containing_target = 0 # This variable stores the probability of the cell x,y containing the target     
    def set_h_value(self, g_x, g_y, heuristic, weight):
        '''This function is used to assign the heuristic value i.e. the h value and the f value to the calling 
           class object. It takes in the argument g_x = goal cell abscissa, g_y = goal cell ordinate, 
           heuristic = It defines the type of heuristic the A* problem is using, weight = this is for problem 
           number 9 (project 1) where we test the effect of weights on the performance of repeated A* and normal A*.
           In this project we have only used the manhattan distance as our heuristic.'''
        if heuristic == 'm': 
            # Manhattan distance
            self.h = weight * abs(g_x - self.x) + abs(g_y - self.y) # calculate the h value
            self.f = self.g + self.h # also updating f inside this function

In [4]:
# Class grid_world -> This class is used to generate a grid world of dimension dim with each cell having 
# probability p of being blocked.
class grid_world:
    def __init__(self,p,dim):
        self.p = p # probability with which the cell is blocked
        self.dim = dim # dimension of the grid (dim x dim)
    def gen_grid(self,matrix):
        '''This function is used to generate the grid. The grid is returned in the form of list of list.'''
        for i in range(self.dim): # looping over the dimension to generate rows of cells
            a = [] # store a row of the grid of len dim
            for j in range(self.dim): # looping over the dim to generate cols of cells
                c = cell(i,j) # cell for coordinates (i,j)
                chance = random.uniform(0,1) 
                # generating a random number between 0 and 1 from uniform distribution, if its less than p 
                # the cell is blocked else the cell is unblocked.
                # if the number generated is less than or equal to p, block the cell.
                if(chance <= self.p):
                    c.blocked = True
                else:
                    # If the cell is unblocked and you are not making the bot_env or the final_disc_gridworld assign the terrain type to the cell.
                    if(self.p!= 1 and self.p!= 0):
                        # To assign the terrain type randomly generate a integer in the range [1,3] and assign terrain corresponding to that number.
                        x = random.randint(1,3) # We use random.randint as there is a equally likely possibility of all three integers coming
                        if(x==1): # If the number is 1 assign terrain flat and false negative rate as 0.2.
                            c.terrain = 'flat' 
                            c.false_neg_rate = 0.2
                        elif x == 2: # If the number is 2 assign terrain hill and false negative rate as 0.5
                            c.terrain = 'hill'
                            c.false_neg_rate = 0.5
                        else: # If the number is 3 assign terrain forest and false negative rate as 0.8
                            c.terrain = 'Forest'
                            c.false_neg_rate = 0.8
                a.append(c) # append the cell to the row
            matrix.append(a) # append the row to the grid
        if(self.p!= 1 and self.p!= 0): # If the grid we are generating is the real world grid
            while(True): # keep randomly choosing start cell and goal cell indices until both lie on unblocked cells.
                start_cell_x = random.randint(0,self.dim-1) # randomly select x coordinate of start cell
                start_cell_y = random.randint(0,self.dim-1) # randomly select y coordinate of start cell
                if matrix[start_cell_x][start_cell_y].blocked == True: # If the selected coordinate is blocked continue 
                    # and reselect the start cell coordinate
                    continue
                target_x = random.randint(0,self.dim-1) # Select the x coordinate of target cell
                target_y = random.randint(0,self.dim-1) # Select the y coordinate of target cell
                if matrix[target_x][target_y].blocked == True: # If the selected coordinate is blocked continue.
                    continue
                break # If both the index lie on unblocked cells break the loop.
            return matrix, (start_cell_x, start_cell_y), (target_x, target_y) # return the final grid, start cell index and goal cell index in case of real_world_grid
        else:
            return matrix # Simply return the grid in case of bot_env and final_disc_gridworld.

#### Helper functions (Agent 6)

In [3]:
# Used by agent 6,7 and 8
def assign_initial_probability(grid):
    '''This function is used to assign the initial belief of probability of containing target in the grid we are 
    working on. It assigns the probability of containing target 1/dim*dim to all the cells in the grid.'''
    num = 1
    den = len(grid)*len(grid)
    for i in range(len(grid)): # looping over all the cells in the grid and assigning probability
        for j in range(len(grid[i])):
            grid[i][j].prob_of_containing_target = num/den

In the following cells we use for loops to calculate the denominator which is used to update the probabilities to mitigate the error of python in storing floting point numbers. Python when storing 0.1 might store 0.1 as 0.100000000000000000065 which might create problem if we directly use the formulas derived in question 2. To mitigate the effect of this error we use for loops to sum over probabilities.

In [5]:
# Used by agent 6,7 and 8
def assign_new_probability_when_target_not_found(grid, cell_x, cell_y):
    '''This function updates the probability of all the cells in the grid when the agents examines a cell and it 
    fails to find target in that cell. The cell examined in (cell_x, cell_y). We pass the grid in which we store 
    the probabilities (bot_env).'''
    FN = grid[cell_x][cell_y].false_neg_rate # False negative rate of the cell examined, used in updating probabilities.
    prob_of_containing_x_y = grid[cell_x][cell_y].prob_of_containing_target # Probability before the cell was examined.
    den = 0 # denominator used in updates.
    # In the following two for loops we calculate the denominator. We take sum of probabilities of all the cells except
    # the cell that was examined, for that cell we multiply the probability with the cells false negative rate before adding the value to den.
    for i in range(len(grid)):
        for j in range(len(grid)):
            if(grid[i][j].blocked): # If the cell is blocked it's prob of containing target is 0 so continue.
                continue
            if (i == cell_x and j == cell_y): # If the cell is the one that was examined add after multiplying with false neg rate.
                den += grid[cell_x][cell_y].prob_of_containing_target * grid[cell_x][cell_y].false_neg_rate
            else: # else directly add the probability to den.
                den += grid[i][j].prob_of_containing_target
    # In the below two for loops we update the probabilities
    for i in range(len(grid)): # looping over all the cells in the grid
        for j in range(len(grid[i])): 
            if(grid[i][j].blocked): # If the cell is blocked the probability will remain 0, so no need to update.
                continue
            else: # If the cell is unblocked
                prob_of_containing = grid[i][j].prob_of_containing_target
                if(i == cell_x and j == cell_y): # If the cell is the one that was examined multiply the probability by false neg rate before dividing by denominator.
                    grid[i][j].prob_of_containing_target = (prob_of_containing_x_y*FN)/den
                else: # else directly divide by denominator.
                    grid[i][j].prob_of_containing_target = prob_of_containing/den

In [6]:
# Used by agent 6,7 and 8
def assign_new_probability_when_new_block_disc(grid, cell_x, cell_y):
    '''This function is used to update the probabilities of all the cells in the grid when the agent encounters a
    blocked cell along its planned path. The (cell_x, cell_y) is the blocked cell.'''
    prob_of_containing_x_y = grid[cell_x][cell_y].prob_of_containing_target # Probability before the cell was found to be blocked
    den = 0 # The den with which we will divide all the probabilities
    for i in range(len(grid)): # looping over all the cells to calculate the denominator.
        for j in range(len(grid)): 
            if(grid[i][j].blocked): # If the cell is blocked continue
                continue 
            if (i == cell_x and j == cell_y): # If the cell is the one found to be blocked continue
                continue
            else: # else add the probability of the cell to the denominator.
                den += grid[i][j].prob_of_containing_target
    for i in range(len(grid)): # looping over all the cells in the grid to update the probabilities.
        for j in range(len(grid[i])):
            if (i== cell_x and j== cell_y): # If the cell is the cell which was found to be blocked make its probability 0.
                grid[i][j].prob_of_containing_target = 0
            else: # else update the probabilities of the cell by dividing by denominator.
                if(grid[i][j].blocked):
                    continue
                else:
                    prob_of_containing = grid[i][j].prob_of_containing_target
                    grid[i][j].prob_of_containing_target = prob_of_containing/den

In [7]:
# Used by agent 6
def choose_next_goal_cell(grid, cell_x, cell_y):
    '''This function is used by the agent to decide the next cell it wants to go to. It is the cell with maximum
    probability of containing target. In case of ties, ties are broken by distance from the cell where your agent 
    is currently standing.'''
    min_dist = math.inf # This is used to break ties by distance in case of tied probabilities.
    target = [] # Stores all the cells with maximum probability of containing the target and having minimum distance from the source.
    max_prob = 0 #Stores the maximum probability
    max_prob_ind = [] #Stores the index of the cells having maximum probability 
    for i in range(len(grid)): # looping over all the cells in the grid.
        for j in range(len(grid[i])):
            if(i == cell_x and j == cell_y): # If it is the current cell continue
                continue
            if grid[i][j].prob_of_containing_target > max_prob: # If the cell has probability more than the max probability
                max_prob = grid[i][j].prob_of_containing_target # update max probability
                max_prob_ind = [[i,j]] # and reassign max prob ind list to [[i,j]] as all the prev stored indices have prob less than the current cell remove those indices from the list.
            elif grid[i][j].prob_of_containing_target == max_prob: # If the cell has same probability as max_probability append the cell to max_prob_ind list.
                max_prob_ind.append([i,j])    
    # The following for loop breaks the ties in probability based on distance, cells with minimum manhattan distance from the source in the list max_prob_ind are stored in target.
    for l in max_prob_ind:
        if abs(cell_x - l[0]) + abs(cell_y - l[1]) < min_dist:
            min_dist = abs(cell_x - l[0]) + abs(cell_y - l[1])
            target = [l]
        elif abs(cell_x - l[0]) + abs(cell_y - l[1]) == min_dist:
            target.append(l)
    return target # return target.

#### Helper function for agent 7 other than the ones shown above in agent 6

In [8]:
def choose_next_goal_cell_agent7(grid, cell_x, cell_y):
    '''This function is used by agent 7 to decide the next cell it wants to go to. It is the cell with the maximum
    probability of successfully finding the target. Ties are broken on distance.'''
    min_dist = math.inf # This is used to break ties by distance in case of tied probabilities.
    target = [] # Stores all the cells with maximum probability of successfully finding the target and having minimum distance from the source.
    max_prob = 0 #Stores the maximum probability
    max_prob_ind = [] #Stores the index of the cells having maximum probability 
    for i in range(len(grid)): # looping over all the cells in the grid.
        for j in range(len(grid[i])):
            if(i == cell_x and j == cell_y): # If it is the current cell continue
                continue
            if grid[i][j].prob_of_containing_target*(1-grid[i][j].false_neg_rate) > max_prob: # If the cell has probability more than the max probability
                max_prob = grid[i][j].prob_of_containing_target*(1-grid[i][j].false_neg_rate) # update max probability
                max_prob_ind = [[i,j]] # and reassign max prob ind list to [[i,j]] as all the prev stored indices have prob less than the current cell remove those indices from the list.
            elif grid[i][j].prob_of_containing_target*(1-grid[i][j].false_neg_rate) == max_prob:
                max_prob_ind.append([i,j])# If the cell has same probability as max_probability append the cell to max_prob_ind list.
    # The following for loop breaks the ties in probability based on distance, cells with minimum manhattan distance from the source in the list max_prob_ind are stored in target.
    for l in max_prob_ind:
        if abs(cell_x - l[0]) + abs(cell_y - l[1]) < min_dist:
            min_dist = abs(cell_x - l[0]) + abs(cell_y - l[1])
            target = [l]
        elif abs(cell_x - l[0]) + abs(cell_y - l[1]) == min_dist:
            target.append(l)
    return target# return target.

#### Helper function for agent 8 other than the ones shown above in agent 6

In [10]:
def choose_next_goal_cell_agent8(grid, cell_x, cell_y):
    '''This function is used by agent 8 to decide the next cell it wants to go to. It is the cell with the maximum
    utility function. Ties are broken on distance.'''
    min_dist = math.inf # This is used to break ties by distance in case of tied probabilities.
    target = [] # Stores all the cells with maximum utility having minimum distance from the source.
    max_utility = 0 # Stores the maximum utility
    max_utility_ind = [] # Stores the index of the cells having maximum utility.
    max_dist = 2*len(grid) # The maximum distance possible in the grid, used to normalize distance while calculating the utility value.
    for i in range(len(grid)): # looping over all the cells in the grid.
        for j in range(len(grid[i])):
            if(i==cell_x and j == cell_y): # If it is the current cell continue.
                continue
            p = grid[i][j].prob_of_containing_target # used in calculating the utility
            FN = grid[i][j].false_neg_rate # used in calculating the utility
            dist = abs(cell_x - i) + abs(cell_y - j) # used in calculating the utility
            # utility is calculated by dividing the probability of successfully finding the target by thr distance of the cell from the agent.
            if p*(1-FN)*max_dist/dist > max_utility: # If the cell has utility value greater than the max utility
                max_utility = p*(1-FN)*max_dist/dist # update max utility
                max_utility_ind = [[i,j]] # and reassign max utility ind list to [[i,j]] as all the prev stored indices have utility less than the current cell remove those indices from the list.
            elif p*(1-FN)*max_dist/dist == max_utility: # If the cell has same utility as max_utility append the cell to max_utility_ind list.
                max_utility_ind.append([i,j])    
    # The following for loop breaks the ties in utility based on distance, cells with minimum manhattan distance from the source in the list max_utility_ind are stored in target.           
    for l in max_utility_ind:
        if abs(cell_x - l[0]) + abs(cell_y - l[1]) < min_dist:
            min_dist = abs(cell_x - l[0]) + abs(cell_y - l[1])
            target = [l]
        elif abs(cell_x - l[0]) + abs(cell_y - l[1]) == min_dist:
            target.append(l)
    return target

#### Function to examine the cell

In [11]:
# Used by all the agents
def find_target_in_goal_cell(grid,cell_x,cell_y,target_cell):
    if((cell_x, cell_y) == target_cell): # If the current cell is the cell in which the target is not present return false
    # else generate a number using random.uniform, if the number is less than or equal to the false negative rate return false 
    # else return true that target was found. This is used to incorporate the false negative rate.
        chance = random.uniform(0,1)
        if chance <= grid[cell_x][cell_y].false_neg_rate:
            return False
        else:
            return True
    else:
        return False

#### Astar

This is the same as previous projects.

In [12]:
def Astar(grid,start_cell_ind,goal_cell_ind, heuristic, weight = 1):
    cells_processed = 0 # number of cells popped from the fringe before reaching the goal.
    number_of_clashes = 0 # number of time we had to update priorities in the fringe.
    start_cell = grid[start_cell_ind[0]][start_cell_ind[1]] # start cell of our grid.
    goal_cell = grid[goal_cell_ind[0]][goal_cell_ind[1]] # goal cell of our grid
    open_list = SortedSet() # fringe, we have used sorted set as our priority queue
    close_list = set() # contains cells already processed, have used set for this purpose (O(1) lookup on average)
    parent_dict = {} # stores parent child relationship, using dictionary for this
    visited = set() # child already visited, using set for this purpose (O(1) lookup on average)
    start_cell.g = 0 # Setting the g value of the start cell to 0
    # setting the h and f value of the start cell using the weight and heuristic passed in the function arguments.
    # weight is for q9
    start_cell.set_h_value(goal_cell_ind[0], goal_cell_ind[1], heuristic, weight)
    open_list.add((start_cell.f, start_cell.h, (start_cell.x, start_cell.y))) # adding the start cell to the priority queue
    # In the queue we are pushing a tuple whose first coordinate is the f value, second coordinate is the h value 
    # then a tuple containing the location of the cell. The priority is set according to the f value in case of a 
    # tie h value is seen.
    visited.add((start_cell.x, start_cell.y)) # updating the visited set as we are pushing the node in priority queue
    while len(open_list)>0: # loop over the priority queue until it is not empty or we reach the goal node
        cl = open_list.pop(0) # pop the node with the least priority (it is at index 0 in the sorted set)
        curr_cell = grid[cl[-1][0]][cl[-1][1]] # the last index in cl contains the x and y coordinates, 
        # storing the node popped from the queue in curr_cell.
        cells_processed += 1 # incrementing the number of cells processed (since we just popped from the queue)
        close_list.add((curr_cell.x, curr_cell.y)) # adding the node to the closed list as it is about to be processed
        if curr_cell == goal_cell: 
            # If the curr cell is the goal cell return the path from the curr cell to goal cell by backtracking 
            # from goal cell to curr cell using the parent dictionary as well as all the nodes 
            # that have been processed uptil now.
            path = []
            curr = curr_cell
            while (curr.x, curr.y) != (start_cell_ind[0], start_cell_ind[1]):
                path.append([curr.x, curr.y])
                curr = parent_dict[(curr.x, curr.y)]
            path.append([curr.x,curr.y])
            return path[::-1], cells_processed, number_of_clashes
        
        children = [] # This list is used to store all the possible valid children of the cell 
        # currently being processed.
        for new_pos in [(0,1),(0,-1),(1,0),(-1,0)]: # 4 main compass directions, field of view also the agent can 
            # only move to these locations
            node_pos = (curr_cell.x+new_pos[0], curr_cell.y+new_pos[1]) # position of the child
            if(node_pos[0] > len(grid)-1 or node_pos[1] > len(grid)-1 or node_pos[0] < 0 or node_pos[1] < 0):
                # if the child is not part of the grid continue
                continue
            if(grid[node_pos[0]][node_pos[1]].blocked):
                # if the child is blocked continue
                continue
            if((node_pos[0], node_pos[1]) in close_list):
                # if child is in close list continue
                continue
            if((node_pos[0], node_pos[1]) in visited):
                # if the child is already visited then compare the previous cost(f) to the new cost(f) and if the
                # new cost is lower update the priority of the child in the priority queue, update the 
                # parent dictionary and update the g value of the node.
                if curr_cell.g + 1 + grid[node_pos[0]][node_pos[1]].h < grid[node_pos[0]][node_pos[1]].f:
                    number_of_clashes += 1 # increment the number of clashes by 1
                    # To update the priority we remove the node from the sorted set (O(logn)) and reinsert with 
                    # new priority O(log(n)). 
                    open_list.remove((grid[node_pos[0]][node_pos[1]].f, grid[node_pos[0]][node_pos[1]].h, (grid[node_pos[0]][node_pos[1]].x, grid[node_pos[0]][node_pos[1]].y)))
                    grid[node_pos[0]][node_pos[1]].g = curr_cell.g+1
                    grid[node_pos[0]][node_pos[1]].set_h_value(goal_cell_ind[0], goal_cell_ind[1], heuristic, weight)
                    open_list.add((grid[node_pos[0]][node_pos[1]].f, grid[node_pos[0]][node_pos[1]].h, (grid[node_pos[0]][node_pos[1]].x, grid[node_pos[0]][node_pos[1]].y)))
                    parent_dict[(node_pos[0],node_pos[1])] = curr_cell # updating the parent dictionary
            else:
                # If the child is not visited update its g value, set its h and f value and add it to the 
                # children list after updating its parent.
                grid[node_pos[0]][node_pos[1]].g = curr_cell.g + 1
                grid[node_pos[0]][node_pos[1]].set_h_value(goal_cell_ind[0], goal_cell_ind[1], heuristic, weight)
                children.append(grid[node_pos[0]][node_pos[1]])
                parent_dict[(node_pos[0],node_pos[1])] = curr_cell
        if(children == []): # If no valid child continue
            continue
        for child in children: # else if valid children exist add them to the open list and update the visited set.
            open_list.add((child.f,child.h, (child.x, child.y)))
            visited.add((child.x, child.y))
    
    return [], 0, 0

#### Explore path functions

##### Agent 6

In [13]:
#function to explore path returned by A* algorithm with limited field of view (the bot can only see in the direction of its movement along the path). 
def explore_path_with_limited_view(path,bot_env ,grid, examine_cost, target_cell_ind, movement_cost):
    '''
    This function helps move the agent along the path returned by A* with unidirectional field of view. The bot environment
    is updated with status of cells along the path traversed. In case the agent reaches the goal cell or the agent encounters
    a blocked cell along it's path probabilities are updated in bot_enviornment and the next target is choosen and returned to
    repeated A* function.
    '''
    x = 0
    y = 0
    for i in range(len(path)): #looping over path list
        x = path[i][0] # x coordinate of the current node
        y = path[i][1] # y coordinate of the current node
        movement_cost += 1 # Increment the movement cost by 1
        #update the bot environment
        bot_env[x][y].x = grid[x][y].x 
        bot_env[x][y].y = grid[x][y].y
        bot_env[x][y].blocked = grid[x][y].blocked
        bot_env[x][y].terrain = grid[x][y].terrain
        bot_env[x][y].false_neg_rate = grid[x][y].false_neg_rate
        if(grid[x][y].blocked): #if the current node is blocked node, return the path traversed till now and update the probabilities.
            assign_new_probability_when_new_block_disc(bot_env,x,y) # Since we discovered a blocked cell, update the probabilities of all the 
            # cells in the bot_enviornment accordingly.
            candidate_targets = choose_next_goal_cell(bot_env,path[i-1][0],path[i-1][1]) # Choose the next target by calling the function
            tar_ind = random.choice(candidate_targets) # randomly choose a index from the list of targets returned above
            new_goal_cell = tar_ind # update the target index
            return path[:i], new_goal_cell, False, examine_cost, movement_cost # return
    # If we reach the goal cell, examine the cell.
    if find_target_in_goal_cell(bot_env,x,y, target_cell_ind): 
        # If examination yealds true return True in second index and increment examination cost by 1.
        examine_cost += 1 
        return path, True, examine_cost, movement_cost
    else:
        # If the examination does not yeild target update the probabilities of all the cells in bot_env accordingly and choose the next goal cell.
        examine_cost += 1 # inc examination cost
        assign_new_probability_when_target_not_found(bot_env, x, y)
        candidate_targets = choose_next_goal_cell(bot_env,x,y)
        tar_ind = random.choice(candidate_targets)
        new_goal_cell = tar_ind
        return path, new_goal_cell ,False, examine_cost, movement_cost

##### Agent 7

In [14]:
#function to explore path returned by A* algorithm with limited field of view (the bot can only see in the direction of its movement along the path). 
def explore_path_with_limited_view_agent_7(path,bot_env ,grid, examine_cost, target_cell_ind, movement_cost):
    '''
    This function helps move the agent along the path returned by A* with unidirectional field of view. The bot environment
    is updated with status of cells along the path traversed. In case the agent reaches the goal cell or the agent encounters
    a blocked cell along it's path probabilities are updated in bot_enviornment and the next target is choosen and returned to
    repeated A* function.
    '''
    x = 0
    y = 0
    for i in range(len(path)): #looping over path list
        x = path[i][0] # x coordinate of the current node
        y = path[i][1] # y coordinate of the current node
        movement_cost += 1 # Increment the movement cost by 1
        #update the bot environment
        bot_env[x][y].x = grid[x][y].x 
        bot_env[x][y].y = grid[x][y].y
        bot_env[x][y].blocked = grid[x][y].blocked
        bot_env[x][y].terrain = grid[x][y].terrain
        bot_env[x][y].false_neg_rate = grid[x][y].false_neg_rate
        if(grid[x][y].blocked): #if the current node is blocked node, return the path traversed till now and update the probabilities.
            assign_new_probability_when_new_block_disc(bot_env,x,y)# Since we discovered a blocked cell, update the probabilities of all the 
            # cells in the bot_enviornment accordingly.
            candidate_targets = choose_next_goal_cell_agent7(bot_env,path[i-1][0],path[i-1][1]) # Choose the next target by calling the function (diff from agent 6)
            tar_ind = random.choice(candidate_targets) # randomly choose a index from the list of targets returned above
            new_goal_cell = tar_ind # update the target index
            return path[:i], new_goal_cell, False, examine_cost, movement_cost # return
    # If we reach the goal cell, examine the cell.
    if find_target_in_goal_cell(bot_env,x,y, target_cell_ind):
        # If examination yealds true return True in second index and increment examination cost by 1.
        examine_cost += 1
        return path, True, examine_cost, movement_cost
    else:
        # If the examination does not yeild target update the probabilities of all the cells in bot_env accordingly and choose the next goal cell.
        examine_cost += 1 # inc the examination cost
        assign_new_probability_when_target_not_found(bot_env, x, y)
        candidate_targets = choose_next_goal_cell_agent7(bot_env,x,y)
        tar_ind = random.choice(candidate_targets)
        new_goal_cell = tar_ind
        return path, new_goal_cell ,False, examine_cost, movement_cost

##### Agent 8

In [15]:
#function to explore path returned by A* algorithm with limited field of view (the bot can only see in the direction of its movement along the path). 
def explore_path_with_limited_view_agent_8(path,bot_env ,grid, final_discovered_grid_world, examine_cost, target_cell_ind, movement_cost):
    '''
    This function helps move the agent along the path returned by A* with unidirectional field of view. The bot environment
    is updated with status of cells along the path traversed. In case the agent reaches the goal cell or the agent encounters
    a blocked cell along it's path probabilities are updated in bot_enviornment and the next target is choosen and returned to
    repeated A* function.
    '''
    x = 0
    y = 0
    for i in range(len(path)): #looping over path list
        x = path[i][0] # x coordinate of the current node
        y = path[i][1] # y coordinate of the current node
        movement_cost += 1 # Increment the movement cost by 1
        #update the bot environment
        bot_env[x][y].x = grid[x][y].x 
        bot_env[x][y].y = grid[x][y].y
        bot_env[x][y].blocked = grid[x][y].blocked
        bot_env[x][y].terrain = grid[x][y].terrain
        bot_env[x][y].false_neg_rate = grid[x][y].false_neg_rate
        if(grid[x][y].blocked): #if the current node is blocked node, return the path traversed till now and update the probabilities.
            assign_new_probability_when_new_block_disc(bot_env,x,y)# Since we discovered a blocked cell, update the probabilities of all the 
            # cells in the enviornment accordingly.
            candidate_targets = choose_next_goal_cell_agent8(bot_env,path[i-1][0],path[i-1][1]) # Choose the next target by calling the function (diff from agent 6)
            tar_ind = random.choice(candidate_targets)  # randomly choose a index from the list of targets returned above
            new_goal_cell = tar_ind # update the target index
            return path[:i], new_goal_cell, False, examine_cost, movement_cost # return
    # If we reach the goal cell, examine the cell.
    if find_target_in_goal_cell(bot_env,x,y, target_cell_ind):
        # If examination yealds true return True in second index and increment examination cost by 1.
        examine_cost += 1
        return path, True, examine_cost, movement_cost
    else:
        # If the examination does not yeild target update the probabilities of all the cells in bot_env accordingly and choose the next goal cell.
        examine_cost += 1
        assign_new_probability_when_target_not_found(bot_env, x, y)
        candidate_targets = choose_next_goal_cell_agent8(bot_env,x,y)
        tar_ind = random.choice(candidate_targets)
        new_goal_cell = tar_ind
        return path, new_goal_cell ,False, examine_cost, movement_cost

#### Repeated A* for Agents 6,7 and 8

In [19]:
# function to implement the A* algorithm
def repeated_astar(bot_env, real_world_grid, start_cell_ind,target_cell_ind, heuristic, weight=1, agent_type = None):
    '''
    This function is the implementation of the repeated A* algorithm. This function calls A* algorithm whenver a 
    blocked cell is encountered along the path returned by A* or the agent examines a cell and is not able to find
    the target. The agent_type flag tells whether the agent is agent 6, 7 or 8.
    '''
    paths = [] # stores the path to be returned.
    #while the A* algorithm doesn't return a path or the maze is found to be not solvable     
    assign_initial_probability(bot_env) # assign initial belief to bot_env (1/dim*dim)
    # The below conditions randomly chooses a neighbour of the current cell to be the target
    if agent_type == 6: 
        goal_cell_ind = random.choice(choose_next_goal_cell(bot_env, start_cell_ind[0], start_cell_ind[1]))
    elif agent_type == 7:
        goal_cell_ind = random.choice(choose_next_goal_cell_agent7(bot_env, start_cell_ind[0], start_cell_ind[1]))
    else:
        goal_cell_ind = random.choice(choose_next_goal_cell_agent8(bot_env, start_cell_ind[0], start_cell_ind[1]))
    examine_cost = 0 # initialize the examine cost to 0
    movement_cost = 0 # initialize the movement cost to 0
    while(1): # keep running until the agent finds the target
        path, cells_processed, n_c = Astar(bot_env, start_cell_ind, goal_cell_ind, heuristic, weight) # call A* with current knowledge of the grid and the start_cell_index and the estimated goal cell.
        if(path == []): # If A* doesn't return a path, path doesn't exist. i.e. the estimated goal cell i.e. the cell with the highest probability is not reachable.
            # Mark the cell as blocked and update the probabilities of all the cells accordingly.
            bot_env[goal_cell_ind[0]][goal_cell_ind[1]].blocked = True
            bot_env[goal_cell_ind[0]][goal_cell_ind[1]].terrain = None
            bot_env[goal_cell_ind[0]][goal_cell_ind[1]].false_neg_rate = 0
            assign_new_probability_when_new_block_disc(bot_env,goal_cell_ind[0],goal_cell_ind[1])
            # Choose the next goal cell depending upon the agent it is. For agent 6 based on probability of containing the target,
            # for agent 7 probability of succcessfully finding the target and for agent 8 the utility function.
            if agent_type == 6:
                # If agent 6 use the choose_next_goal_cell function to choose the next goal cell.
                candidate_targets = choose_next_goal_cell(bot_env,start_cell_ind[0],start_cell_ind[1])
                tar_ind = random.choice(candidate_targets)
                goal_cell_ind = tar_ind
                continue
            elif agent_type == 7:
                # If agent 7 use choose_next_goal_cell_agent7 to choose the next goal cell.
                candidate_targets = choose_next_goal_cell_agent7(bot_env,start_cell_ind[0],start_cell_ind[1])
                tar_ind = random.choice(candidate_targets)
                goal_cell_ind = tar_ind
                continue
            else:
                # If agent 8 use choose_next_goal_cell_agent8 to choose the next goal cell.
                candidate_targets = choose_next_goal_cell_agent8(bot_env,start_cell_ind[0],start_cell_ind[1])
                tar_ind = random.choice(candidate_targets)
                goal_cell_ind = tar_ind
                continue
        else:  # If path exists, i.e. the target is reachable:
            if agent_type == 6:
                # If it is agent 6 call explore_path_with_limited_view
                params = explore_path_with_limited_view(path,bot_env,real_world_grid, examine_cost, target_cell_ind, movement_cost) #explore path with limited view
            elif agent_type == 7:
                # If it is agent 7 call explore_path_with_limited_view_agent_7
                params = explore_path_with_limited_view_agent_7(path,bot_env,real_world_grid, examine_cost, target_cell_ind, movement_cost) #explore path with limited view
            else:
                # If it is agent 8 call explore_path_with_limited_view_agent_8
                params = explore_path_with_limited_view_agent_8(path,bot_env,real_world_grid, final_discovered_grid_world, examine_cost, target_cell_ind, movement_cost) #explore path with limited view
            p = params[0] # Store the path travelled by the agent in p
            paths.append(p)  # append the path returned to paths
            if(params[1] == True): # If the agent was able to find the target
                examine_cost = params[2] # update examination cost
                movement_cost = params[3] # update movement cost
                return paths, examine_cost, movement_cost # return
            else: # If the agent was not able to find the target
                start_cell_ind = p[-1] # update the start cell index
                goal_cell_ind = params[1] # update the goal cell index
                examine_cost = params[3] # update examination cost
                movement_cost = params[4]-1 # update movement cost

#### Appendix for Agent 9

In [20]:
# function to implement the A* algorithm
def repeated_astar_agent9(bot_env, real_world_grid, start_cell_ind,target_cell_ind, heuristic, weight=1):
    '''
    This function is the implementation of the repeated A* algorithm. This function calls A* algorithm whenver a 
    blocked cell is encountered along the path returned by A* or the agent examines a cell and is not able to find
    the target.
    '''
    paths = [] # stores the path to be returned.     
    track_states = [] # Used to detect oscillations as described in report
    belief = gen_initial_belief_matrix(len(bot_env)) # generate initial belief matrix.
    candidate_targets = choose_next_goal_cell_agent9(bot_env, start_cell_ind[0], start_cell_ind[1], belief) # choose a estimated target other than the start cell
    tar_ind = random.choice(candidate_targets)
    goal_cell_ind = tar_ind
    examine_cost = 0 # initialize the examination cost to 0
    movement_cost = 0 # initialize the movement cost to 0
    do_not_move = False # flag to make the target stop moving, used in the case when in planning the agent finds that the estimated target is not reachable in that case
    # we don't want the target to move.
    sensed = False # variable to tell if the agent was able to sense the target or not.
    prev_start_cell_ind = start_cell_ind # Used to store the previous start cell index as the agent encounters a blocked cell it has to go back to the previous start cell.
    while(1): # Run until the agent finds the target
        count += 1 # used for the first iteration don't want the target to move initially when for the first time we call repeated A* and the agent just stats it's exploration
        if(count != 1): 
            # If the do_not_move flag is set don't move the target else move the target as the agent moves.
            if do_not_move:
                do_not_move = False
            else:
                target_cell_ind = move_target(real_world_grid,target_cell_ind[0], target_cell_ind[1])
        # append the start_cell_ind and goal_cell_ind to the track states.
        track_states.append((start_cell_ind, goal_cell_ind))
        # If the length of trackstates is more than 20 and the agent is oscillating between two states, randomly place the agent in one of the neighbours to make it stop oscillating.
        if(len(track_states)>=20):
            flag = True 
            # If the agent is not oscillating between two states make the flag false.
            for i in range(len(track_states)-2):
                if(track_states[i] != track_states[i+2]):
                    flag = False
            if(flag):
                start_cell_ind = move_agent_randomly(start_cell_ind) # move the agent randomly
                movement_cost += 1 # increment movement cost
                target_cell_ind = move_target(real_world_grid, target_cell_ind[0], target_cell_ind[1]) # move the target as agent moved.
                continue
            track_states =[] # make the track states empty (saving space).
        path, cells_processed, n_c = Astar(bot_env, start_cell_ind, goal_cell_ind, heuristic, weight) # call A* with current knowledge of the grid
        if(path == []): # If A* doesn't return a path, path doesn't exist. i.e. the estimated goal cell i.e. the cell with the highest probability is not reachable.
            # Mark the cell as blocked and update the probabilities of all the cells accordingly.
            bot_env[goal_cell_ind[0]][goal_cell_ind[1]].blocked = True
            bot_env[goal_cell_ind[0]][goal_cell_ind[1]].terrain = None
            bot_env[goal_cell_ind[0]][goal_cell_ind[1]].false_neg_rate = 0
            assign_new_probability_when_new_block_disc_agent9(bot_env,belief,goal_cell_ind[0],goal_cell_ind[1])
            candidate_targets = choose_next_goal_cell_agent9(bot_env,start_cell_ind[0],start_cell_ind[1], belief)
            tar_ind = random.choice(candidate_targets)
            goal_cell_ind = tar_ind
            do_not_move = True # set the do not move flag to true we dont want the target to move as agent did not move it is still in planning stage
            continue    
        else:  # If path exists, do this:
            # Call the explore_path function to move to the next cell along the planned path.
            params = explore_path_with_limited_view_agent9(path,bot_env,real_world_grid,examine_cost,target_cell_ind, movement_cost, belief, prev_start_cell_ind)
            p = params[0] # p is the path returned by explore_path function. Always length 2 in case of agent 9 as we always move a step and update probabilities and change the goal cell.
            paths.append(p)  # append the path returned to paths  
            belief = params[-2] # update the belief with the belief returned by explore path function
            sensed = params[-3] # Update the sensed variable.
            prev_start_cell_ind = params[-1] # update previous start cell index.
            if (params[1] == True): # If the agent encountered a blocked cell along the path restart from the cell we reached that cell.
                start_cell_ind = prev_start_cell_ind
            else: # If blocked cell was not encounter the start cell is the cell on which the agent is present.
                start_cell_ind = p[-1]
            goal_cell_ind = params[2] # Update the goal cell
            examine_cost = params[3] # update examination cost
            if sensed == False and params[1] == False:
                movement_cost = params[4] # If the agent not sense the target and the cell it went to was not blocked update the movement cost
            if sensed == True: # If the agent sensed the target
                while(sense_target(target_cell_ind, start_cell_ind)): # We keep the agent stationary until the target is in its neighbourhood as soon as the agent fails to sense the target 
                    # we examine the cell it is currently standing in.
                    target_cell_ind = move_target(real_world_grid, target_cell_ind[0], target_cell_ind[1]) # keep on moving the target but keep the agent stationary
                    continue
                examine_cost += 1 # increment examination cost
                # If the agent finds the target in the goal cell return
                if(find_target_in_goal_cell(bot_env, start_cell_ind[0], start_cell_ind[1], target_cell_ind)):
                    return paths, examine_cost, movement_cost
                else:
                    # If it fails to find the target, the target left the agents neighbourhood make the sensed variable false
                    sensed = False
                    do_not_move = True # set do not move flag to true as it will cause the target to take 2 steps when agent did not take even one step if we dont set this to true
                    belief = predict_one_step(bot_env, belief) # update the belief
                    candidate_targets = choose_next_goal_cell_agent9(bot_env,start_cell_ind[0],start_cell_ind[1], belief) # choose the next goal cell
                    tar_ind = random.choice(candidate_targets)
                    new_goal_cell = tar_ind # update the goal cell

In [21]:
#function to explore path returned by A* algorithm with limited field of view (the bot can only see in the direction of its movement along the path). 
def explore_path_with_limited_view_agent9(path, bot_env, grid, examine_cost, target_cell_ind, movement_cost, belief, previous_start_cell):
    '''
    This function helps move the agent along the path returned by A* with unidirectional field of view. The bot environment
    is updated with status of cells along the path traversed. The agent always takes one step in this function and after that step
    returns to repeated A* function where the rest of processing is done.
    '''
    x = 0
    y = 0
    sensed = False # Initialize the sensed variable to false
    for i in range(len(path)): #looping over path list                
        x = path[i][0] # x coordinate of the current node
        y = path[i][1] # y coordinate of the current node
        movement_cost += 1 # Increment the movement cost
        #update the bot environment
        bot_env[x][y].x = grid[x][y].x 
        bot_env[x][y].y = grid[x][y].y
        bot_env[x][y].blocked = grid[x][y].blocked
        bot_env[x][y].terrain = grid[x][y].terrain
        bot_env[x][y].false_neg_rate = grid[x][y].false_neg_rate
        
        if(grid[x][y].blocked): #if the current node is blocked node, return the path traversed till now, update the belief and choose the new goal cell.
            belief = assign_new_probability_when_new_block_disc_agent9(bot_env,belief,x,y)
            candidate_targets = choose_next_goal_cell_agent9(bot_env,path[i-1][0],path[i-1][1], belief)
            tar_ind = random.choice(candidate_targets)
            new_goal_cell = tar_ind
            # return true in second index to tell the repeated a* function that we found a block
            # return the previous start cell as the parent of this cell which was passed when this function was called.
            return path[:i], True ,new_goal_cell, examine_cost, movement_cost, sensed, belief, previous_start_cell
        # If the next cell along the planned path is not blocked
        if(sense_target(target_cell_ind, (x,y)) == False):
            # If the agent did not sense the target, we change the belief of all the neighbouring cells to 0 by calling the function update_prob
            # and passing the second last argument i.e. in_vicinity as false.
            belief = update_prob(bot_env, belief, False, (x,y))
            predict = predict_one_step(bot_env, belief) # make predictions about the target using the updated belief
            belief = predict # change the belief for time (t+1)
            candidate_targets = choose_next_goal_cell_agent9(bot_env,path[i][0],path[i][1], belief) # choose the next goal cell from the belief at time (t+1)
            tar_ind = random.choice(candidate_targets)
            new_goal_cell = tar_ind # update the new goal cell and return this
            sensed = False # make the sensed variable false
            return path[:i+2], False ,new_goal_cell, examine_cost, movement_cost, sensed, belief, path[:i+1][0] # return
        else:
            # If the agent senses the target in its neighbourhood make the sensed variable flag true, change the belief accordingly.
            belief = update_prob(bot_env, belief, True, (x,y)) # update the belief
            predict = predict_one_step(bot_env, belief) # make predictions
            belief = predict # set the belief at time (t+1) without t+1 observation to predict
            candidate_targets = choose_next_goal_cell_agent9(bot_env,path[i][0],path[i][1], belief) # choose new goal cell using that belief
            tar_ind = random.choice(candidate_targets)
            new_goal_cell = tar_ind
            sensed = True
            return path[:i+1], False ,new_goal_cell, examine_cost, movement_cost, sensed, belief, previous_start_cell # return

In [26]:
def move_agent_randomly(start_cell_ind):
    '''This function is used to move the agent randomly to one of its unblocked neighbours in case it is stuck in 
    oscillation.'''
    node_pos = [(-1,0), (0,-1), (1,0), (0,1)]
    children = []
    for i in node_pos:
        x_idx = start_cell_ind[0]+i[0]
        y_idx = start_cell_ind[1]+i[1]
        if(x_idx<len(real_world_grid) and x_idx>=0 and y_idx<len(real_world_grid) and y_idx>=0):
            if(real_world_grid[x_idx][y_idx].blocked == False):
                children.append((x_idx, y_idx))
    idx = random.randint(0, len(children)-1)
    return children[idx]

Update probability when new block is discovered and choose next goal cell functions are same as agent 8 (update probability function of agent 6 and choose_next_goal_cell_agent8), instead of choosing the probability stored in bot_env we use the belief matrix that we have generated.

In [25]:
def assign_new_probability_when_new_block_disc_agent9(grid,belief, cell_x, cell_y):
    prob_of_containing_x_y = belief[cell_x][cell_y]
    den = 0
    
    for i in range(len(grid)):
        for j in range(len(grid)):
            if(grid[i][j].blocked):
                continue
            if(i==cell_x and j == cell_y):
                continue
            else:
                den += belief[i][j]
    
    for i in range(len(grid)):
        for j in range(len(grid[i])):
            if (i== cell_x and j== cell_y):
                belief[i][j] = 0
            else:
                if(grid[i][j].blocked):
                    continue
                else:
                    belief[i][j]= belief[i][j]/den
    return belief

In [24]:
def choose_next_goal_cell_agent9(grid, cell_x, cell_y, belief):
    min_dist = math.inf
    target = []
    max_utility = 0
    max_utility_ind = []
    max_dist = 200
    for i in range(len(grid)):
        for j in range(len(grid[i])):
            if(i==cell_x and j == cell_y):
                continue
            p = belief[i][j]
            FN = grid[i][j].false_neg_rate
            dist = abs(cell_x - i) + abs(cell_y - j)
            if p*(1-FN)*max_dist/dist > max_utility:
                max_utility = p*(1-FN)*max_dist/dist
                max_utility_ind = [[i,j]]
            elif p*(1-FN)*max_dist/dist == max_utility:
                max_utility_ind.append([i,j])    

    for l in max_utility_ind:
        if abs(cell_x - l[0]) + abs(cell_y - l[1]) < min_dist:
            min_dist = abs(cell_x - l[0]) + abs(cell_y - l[1])
            target = [l]
        elif abs(cell_x - l[0]) + abs(cell_y - l[1]) == min_dist:
            target.append(l)
    return target

In [27]:
def move_target(real_world_grid, x, y):
    '''This function is used to move the target to one of its unblocked neighbours at random.'''
    node_pos = [(-1,0), (0,-1), (1,0), (0,1)]
    children = []
    for i in node_pos:
        x_idx = x+i[0]
        y_idx = y+i[1]
        if(x+i[0]<len(real_world_grid) and x+i[0]>=0 and y+i[1]<len(real_world_grid) and y+i[1]>=0):
            if(real_world_grid[x_idx][y_idx].blocked == False):
                children.append((x+i[0], y+i[1]))
    idx = random.randint(0, len(children)-1)
    return children[idx]

In [28]:
def sense_target(target, agent):
    '''This function is used by the agent to sense for the target in its neighbourhood.'''
    if(abs(target[0]-agent[0])<=1 and abs(target[1]-agent[1])<=1 and target!=agent):
        # if the diff between the x coordinate or the y coordinate is <= 1 means that the target is either on the 
        # agent cell or in its neighbourhood.
        if(abs(target[0]-agent[0])==0 and abs(target[1]-agent[1])==0): # if its on the agent cell return false.
            return False
        return True # else return true.
    return False # if diff is more than 1 return false.

In [29]:
def gen_initial_belief_matrix(dim):
    '''This function is used to get the initial belief matrix.'''
    belief = [] 
    for i in range(dim):
        row = []
        for j in range(dim):
            prob = 1/(dim*dim) # assign the value of 1/dim*dim to all the cells in the belief matrix.
            row.append(prob)
        belief.append(row)
    return belief

In [30]:
def count_unblocked_neighbours(bot_env, pos):
    '''This function is used to count the unblocked neighbours of a given cell. Helpful in making the prediction 
    for time t+1.'''
    node_pos = [(-1,0), (0,-1), (1,0), (0,1)]
    children = []
    cnt = 0
    for i in node_pos:
        x_idx = pos[0]+i[0]
        y_idx = pos[1]+i[1]
        if(x_idx<len(real_world_grid) and x_idx>=0 and y_idx<len(real_world_grid) and y_idx>=0):
            if(bot_env[x_idx][y_idx].blocked == False):
                cnt+=1
    return cnt

In [32]:
def predict_one_step(bot_env, belief):
    '''This function is used to make the prediction for target\'s location using the belief matrix passed.'''
    predict = [] # initialize the predict matrix.
    for i in range(len(belief)): # loop over all the cells in the belief matrix
        a = [] # used to store the row of predictions
        for j in range(len(belief)): 
            pred = 0 # initialize prediction value for cell (i,j)
            if(bot_env[i][j].blocked): # If the cell is blocked don't make pred for that cell
                a.append(pred)
                continue
            neighbours = [(0,1), (1,0), (0,-1), (-1,0)] # looping over all the neighbours of the cell (i,j)
            for node in neighbours:
                # if the neighbour is a valid neighbour
                if(node[0]+i < len(bot_env) and node[0]+i >= 0 and node[1]+j >=0 and node[1]+j < len(bot_env)):
                    if(bot_env[node[0]+i][node[1]+j].blocked): # if the neighbour is blocked continue
                        continue
                    neighbour = (node[0]+i, node[1]+j)
                    num_of_unblocked = count_unblocked_neighbours(bot_env, neighbour) # count the number of unblocked cells of this neighbour as its 
                    # probability will be distributed equally among these neighbours
                    if (num_of_unblocked == 0): # if all neighbours are blocked no changes to pred
                        pred += 0
                    else:
                        # else update the prediction.
                        pred += 1/(num_of_unblocked) * belief[neighbour[0]][neighbour[1]]
            a.append(pred) # append the prediction in row
        predict.append(a) # append the row in pred matrix
    return predict

In [33]:
def update_prob(bot_env, belief, in_vicinity, pos):
    '''This function is used by agent 9 to update its belief matrix in the cases:
    (1) agent did not sense target in its surrounding
    (2) agent sensed target in its surrounding'''
    prob_total = 0 # sum of probabilities of all the neighbours, we update the probability of all the cells except these cells by dividing by (1-this sum).
    children = []
    if(in_vicinity == False): # If the target is not in the vicinity of the agent
        node_pos = [(0,1),(1,0), (0,-1), (-1,0), (1,1), (-1,-1), (1,-1), (-1,1)]
        for node in node_pos:
            if(node[0]+pos[0] < len(bot_env) and node[0]+pos[0] >= 0 and node[1]+pos[1] >= 0 and node[1]+pos[1] < len(bot_env) and bot_env[node[0]+pos[0]][node[1]+pos[1]].blocked == False):
                # If the neighbour is a valid and unblocked neighbour add its probability to prob_total
                prob_total += belief[node[0]+pos[0]][node[1]+pos[1]]
                children.append((node[0]+pos[0], node[1]+pos[1]))
                belief[node[0]+pos[0]][node[1]+pos[1]] = 0 # make its belief to 0
        for i in range(len(belief)): # update the belief of all other cells by dividing by (1-prob_total)
            for j in range(len(belief)):
                if((i,j) not in children and bot_env[i][j].blocked == False):
                    belief[i][j] = belief[i][j]/(1-prob_total)
    else: # If the target is in the vicinity of the agent
        node_pos = [(0,1),(1,0), (0,-1), (-1,0), (1,1), (-1,-1), (1,-1), (-1,1)]
        children = [] # make a children list
        for node in node_pos:
            if(node[0]+pos[0] < len(bot_env) and node[0]+pos[0] >= 0 and node[1]+pos[1] >= 0 and node[1]+pos[1] < len(bot_env) and bot_env[node[0]+pos[0]][node[1]+pos[1]].blocked == False):
                # add all valid unblocked cells to this list
                children.append((node[0]+pos[0], node[1]+pos[1]))
        number_of_children = len(children)
        for i in range(len(belief)):
            for j in range(len(belief)):
                if((i,j) not in children and bot_env[i][j].blocked == False):
                    belief[i][j] = 0 # if the cell is not in the children list make its belief to 0
                elif((i,j) in children):
                    # if the cell is in the children list make its belief to 1/number of children.
                    belief[i][j] = 1/number_of_children
    return belief # return belief matrix